In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/disaster-tweets-data/sample_submission.csv
/kaggle/input/disaster-tweets-data/train.csv
/kaggle/input/disaster-tweets-data/test.csv


<h3>Import Libraries</h3>

In [2]:
import re
import nltk
import torch
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, DataCollatorWithPadding
from torch.utils.data import DataLoader, Dataset

2025-07-18 00:33:24.931058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752798805.175427      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752798805.261630      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<h3>Get Data</h3>

In [3]:
train_data = pd.read_csv('/kaggle/input/disaster-tweets-data/train.csv')
test_data = pd.read_csv('/kaggle/input/disaster-tweets-data/test.csv')

In [4]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
test_data['target'] = False
data = pd.concat([train_data, test_data], sort=False, ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10876 entries, 0 to 10875
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        10876 non-null  int64 
 1   keyword   10789 non-null  object
 2   location  7238 non-null   object
 3   text      10876 non-null  object
 4   target    10876 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 425.0+ KB


<h3>Cleaning Location Feature</h3>

In [7]:
data['location'] = data['location'].fillna('Unknown')

In [8]:
# lowercase text
data['location_clean'] = data['location'].str.lower()

# remove any non words
data['location_clean'] = data.apply(lambda x: re.sub(r'([^\w\s])', '', x['location_clean']), axis=1)

In [9]:
# extracts text and ner tag for each token used for the location of each tweet
def get_ner_vals(doc):
    return [(token.text, token.label_) for token in doc.ents]

In [10]:
# Normalizes the labels to only 5 potential labels
def change_label_name(label):
    LOCATION_LABELS = ["GPE", "LOC", "FAC"]
    
    if label == "ORG" or label == "PERSON":
        return label
    elif label in LOCATION_LABELS:
        return "LOCATION" 
    elif label:
        return "OTHER"
    return "NONE"

In [11]:
# gets the most frequent label from the tokens to declare the phrase as that label
def get_most_freq_label(ent_labels):
    if len(ent_labels) == 0:
        return "NONE"
    labels_series = pd.Series(ent_labels)
    return labels_series.value_counts().index[0]

In [12]:
nlp = spacy.load('en_core_web_sm')
docs = list(nlp.pipe(data['location'].tolist(), disable=[])) # process the docs for each locaiton value
data['ner_ents'] = [get_ner_vals(doc) for doc in docs]

# extract labels from NER entities
data['ner_labels'] = data['ner_ents'].apply(lambda ents: [change_label_name(label) for _, label in ents])

# use the most frequent NER label as the label for all the tokens
data['location_ner'] = data['ner_labels'].apply(get_most_freq_label)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10876 entries, 0 to 10875
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              10876 non-null  int64 
 1   keyword         10789 non-null  object
 2   location        10876 non-null  object
 3   text            10876 non-null  object
 4   target          10876 non-null  int64 
 5   location_clean  10876 non-null  object
 6   ner_ents        10876 non-null  object
 7   ner_labels      10876 non-null  object
 8   location_ner    10876 non-null  object
dtypes: int64(2), object(7)
memory usage: 764.8+ KB


In [14]:
data['location_ner'].value_counts()

location_ner
NONE        5153
LOCATION    4016
ORG          946
PERSON       507
OTHER        254
Name: count, dtype: int64

<h5>Now, we will add the location to the text to add more context to the tweet.</h5>

In [15]:
def add_location_and_text(text, location, ner_tag):
    if ner_tag == 'LOCATION' or ner_tag == "ORG":
        return text + ', [LOC] ' + location
    else:
        return text

In [16]:
# lowercase
data['text_clean'] = data['text'].str.lower()
data['location_clean'] = data['location'].str.lower()

# remove any non words
data['location_clean'] = data.apply(lambda x: re.sub(r'([^\w\s])', '', x['location_clean']), axis=1)

# add both columns together if location is good
data['text + location'] = data.apply(lambda x: add_location_and_text(x['text_clean'], x['location_clean'], x['location_ner']), axis=1)
data.head()

,id,keyword,location,text,target,location_clean,ner_ents,ner_labels,location_ner,text_clean,text + location
0,1,NaN,Unknown,Our Deeds are the Reason of this #earthquake M...,1,unknown,[],[],NONE,our deeds are the reason of this #earthquake m...,our deeds are the reason of this #earthquake m...
1,4,NaN,Unknown,Forest fire near La Ronge Sask. Canada,1,unknown,[],[],NONE,forest fire near la ronge sask. canada,forest fire near la ronge sask. canada
2,5,NaN,Unknown,All residents asked to 'shelter in place' are ...,1,unknown,[],[],NONE,all residents asked to 'shelter in place' are ...,all residents asked to 'shelter in place' are ...
3,6,NaN,Unknown,"13,000 people receive #wildfires evacuation or...",1,unknown,[],[],NONE,"13,000 people receive #wildfires evacuation or...","13,000 people receive #wildfires evacuation or..."
4,7,NaN,Unknown,Just got sent this photo from Ruby #Alaska as ...,1,unknown,[],[],NONE,just got sent this photo from ruby #alaska as ...,just got sent this photo from ruby #alaska as ...


In [17]:
data['text + location'][0]

'our deeds are the reason of this #earthquake may allah forgive us all'

<h3>Text Preprocessing</h3>

In [18]:
nltk.download('stopwords')
en_stopwords = stopwords.words('english')
print(en_stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# remove links from the text
data['text_loc_clean'] = data.apply(lambda x: re.sub(r'http\S+|www\.\S+', '', x['text + location']), axis=1)

# remove any special character or non words
data['text_loc_clean'] = data.apply(lambda x: re.sub(r'([^\w\s])', "", x['text_loc_clean']), axis=1)

# remove stopwords
data['text_loc_clean'] = data['text_loc_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (en_stopwords)]))



train_tweets = data['text_loc_clean'][:train_data.shape[0]].copy().tolist()
train_labels = data['target'][:train_data.shape[0]].copy().tolist()
test_tweets = data['text_loc_clean'][train_data.shape[0]:].copy().tolist()

<h5>Tokenizes them by returning input ids that come from the vocabulary of the BERT model being used. Attention Mask shows what is padding and what isn't (1 for actual words, 0 for padding).</h5>

In [20]:
train_X, val_X, train_y, val_y = train_test_split(train_tweets, train_labels, test_size=0.2)

In [21]:
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the tweets (using pt tensors since we are using PyTorch and Hugging Face BERT models)
# they require the input to be the same tensor type when loading model (DL framework provided by hugging face)
train_encodings = tokenizer(train_X, padding=True, truncation=True, return_tensors="pt")
val_encodings = tokenizer(val_X, padding=True, truncation=True, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

<h3>Create Custom Dataset</h3>

<h5>Need to create dataset to use for GPU optimization so that the GPU does not have to wait for the CPU to process the batches so they can be trained on the GPU. CPU works in parallel with the GPU. GPU handles to computation like matrix multiplication and forward pass and back prop, while CPU handles the data preprocessing and loading into batches for the GPU to use for those computations.</h5>

<h5>Also, batches contain a certain number of sample that represent each tweet along with its target label. DataLoader is able to handle all of this.</h5>

In [22]:
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # tells you how many sample are in your dataset (needed by DataLoader to know
    # when an epoch (one pass through dataset) ends)
    # an update per batch (backprop)
    def __len__(self):
        return len(self.labels)

    # returns one sample using its index (a single data point)
    # used by DataLoader to create batches
    # goes through input ids and attention masks to create a dictionary that describes the sample
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [23]:
train_dataset = TweetDataset(train_encodings, train_y)
val_dataset = TweetDataset(val_encodings, val_y)

<h3>Model Finetuning</h3>

<h5>Need training arguments to tell Trainer how to train your model.
Its like a recipe for training: batch size, number of epochs, where to save results, etc.</h5>

In [24]:
# training_args = TrainingArguments(
#     output_dir="/kaggle/working/results",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     evaluation_strategy="epoch", # when to evaluate and update framework
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     logging_steps=50,
#     fp16=torch.cuda.is_available(),  # mixed precision if supported
# )

In [25]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=1)
#     return {
#         "f1": f1_score(labels, preds, average="weighted")
#     }

In [26]:
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=data_collator,
#     tokenizer=tokenizer,          # optional but helps with logging & saving
#     compute_metrics=compute_metrics,
# )

# trainer.train()
# eval_results = trainer.evaluate()
# print("Results: ", eval_results)

<h3>Model Testing</h3>